In [ ]:
from bs4 import BeautifulSoup as soup
import requests as req 

# Mangafy 

---

### Parse name 
Parse a string removing and replacing unwanted characters

In [ ]:
def parse_name(name):
	print("Parsing {}".format(name))
	name_to_return = name
	allowed_characters = "QWERTYUIOPLKJHGFDSAZXCVBNMqwertyuioplkjhgfdsazxcvbnm-_"
	for x in name_to_return:
		if x in allowed_characters:
			continue
		if x == '&':
			name_to_return = name_to_return.replace(x, 'and')
		name_to_return = name_to_return.replace(x, ' ')
	print("Result: {}".format(name_to_return))
	return name_to_return

#test
tesr_string = 'jwefnw44ouh034jtrh380th3obg0934tp24[pri0394utop3mtoih3490tj23rnpi2h3r[j23pitgp2o34jtoi34ht[pk'
print(parse_name(tesr_string))

---

### Manga Genre Scrapper 
Scrape the first ten manga from the specified url (must be a valid genres page on manganelo.com)
> Example **https://manganelo.com/genre-all?type=newest**

In [ ]:
manga_list = []

def genre_scrape(url):
    manga_list.clear()
    """WARNING: url must be a valid url to one of the genres page on manganelo.com"""    
    uClient = req.get(url)
    client_html = uClient.text
    client_soup = soup(client_html, 'lxml')
    homepage_items = client_soup.findAll("div",{"class":"content-genres-item"})
    print('Found {} manga'.format(len(homepage_items)))
    index = 0

    for homepage_item in homepage_items:
        try:
            manga_name = parse_name(homepage_item.div.h3.a.text)
            latest_chapter_box = homepage_item.find("a",{"class":"genres-item-chap text-nowrap a-h"})
            if latest_chapter_box == None:
                latest_chapter = "Unable to fetch latest chapter"
            else:
                latest_chapter = latest_chapter_box.text
            manga_link = homepage_item.div.h3.a["href"]

            manga_list.append({"Name":manga_name, "Link":manga_link, "Latest_chapter":latest_chapter})

            index = index + 1
            if index == 10:
                break
        except UnboundLocalError:
            print("local variable 'index' referenced before assignment")
        except:
            print('An error occoured')

    return manga_list

#test
test_url = 'https://manganelo.com/genre-all?type=newest'
print(genre_scrape(test_url))

---

### Get Manga Data 
Get the manga data from a list of alerts
>```
>#alert list sample
>{
>    'manga1': 'manga1_link',
>    'manga2': 'manga2_link',
>    'manga3': 'manga3_link'
>} 
>```

And returns it in the following format
>```
>[
>    {'name': 'manga1', 'link': 'manga1_link', 'latest_chapter': 'latest_chapter'},
>    {'name': 'manga2', 'link': 'manga2_link', 'latest_chapter': 'latest_chapter'},
>    {'name': 'manga3', 'link': 'manga3_link', 'latest_chapter': 'latest_chapter'}
>] 
>```

In [ ]:
def show_updates(alert_list):

    manga_list = []

    for x in alert_list:

        uClient = req.get(alert_list[x])
        client_html = uClient.text
        client_soup = soup(client_html, 'lxml')

        homepage_item = client_soup.find("li", {"class": "a-h"})

        latest_chapter = homepage_item.a.text
        latest_chapter_link = homepage_item.a['href']

        manga_list.append(
            {"name": x, "link": alert_list[x], "latest_chapter": latest_chapter, "latest_chapter_link": latest_chapter_link})

    return manga_list